In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai_tools import PDFSearchTool, SerperDevTool, ScrapeWebsiteTool
from crewai import Agent, Task, Crew, Process, LLM

In [2]:
load_dotenv('.env')

SERPER_API_KEY = os.getenv("SERPER_API_KEY")

llm = LLM(
    model="openai/"+os.getenv("OPENAI_MODEL_NAME"), 
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://litellm.govtext.gov.sg/",
    default_headers={"user-agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/81.0"}
)

In [3]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

You entered: baby bonus


In [4]:
tool_pdf = PDFSearchTool(
    pdf=r'C:\Users\limya\Documents\AI Bootcamp\ABC_Project\audit_buddy\Files\AR_2024.pdf',
    config=dict(
        embedder=dict(
            provider="openai",
            config=dict(
                model="text-embedding-3-large-prd-gcc2-lb"
            ),
        ),
    )
)

2024-10-13 13:17:33,317 - 18260 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 295 0 (offset 0)
2024-10-13 13:17:33,318 - 18260 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 669 0 (offset 0)
2024-10-13 13:17:33,319 - 18260 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 681 0 (offset 0)
Inserting batches in chromadb: 100%|██████████| 3/3 [00:15<00:00,  5.32s/it]


In [5]:
# tool_pdf = PDFSearchTool(pdf=r'C:\Users\limya\Documents\AI Bootcamp\ABC_Project\audit_buddy\Files\AR_2024.pdf')
# tool_search = SerperDevTool(n_results=5, country="sg")
# tool_webscrape = ScrapeWebsiteTool()

In [6]:
# auditor = Agent(
#     role = "auditor",
#     goal="Return the Audit Planning Memo on {topic} based on inputs from researcher and past audit findings from the audit assistant.",
#     backstory = """You are an auditor. You need to consider the inputs from the researcher and audit assistant to determine what to write for the audit planning memo and risk assessment""",
#     allow_delegation = True,
#     max_iter = 15,
#     verbose = True,
#     llm=llm
# )

# researcher = Agent(
#     role="researcher",
#     goal="From the internet, research and analyse and sieve out information related to {topic} only." ,
#     backstory="""As a researcher, navigating and extracting critical information is crucial. You are assisting the auditor to research on topics relevant to {topic}. 
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter = 15,
#     llm=llm
# )

# Audit_asssitant = Agent(
#     role = "Audit assistant",
#     goal ="From the file provided, extract the audit findings pertaining to {topic} only. Pass the audit findings to auditor",
#     backstory = "You are an audit assistant. Your job is to extract the past audit findings relating to the {topic} and pass it to the auditor.",
#     allow_delegation=False,
#     verbose=True,
#     max_iter = 15,
#     llm=llm
# )

# task_search = Task(
#     description="""Returns a JSON list of websites that contains the top 5 most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only. Ignore urls that ends with .pdf.
#     """,
#     expected_output="""\
#     Returns websites that contain the top 5 most relavant policy and regulation information on {topic} only. The output must be in JSON format with the description as the key and url as value.""",

#     agent=researcher,
#     tools=[tool_search],
# )


# task_websitesearch = Task(
#     description="""Scrape through a website for information related to {topic} only
#     """,
#     expected_output="""\
#     Summarise the website for information related to {topic} only in the Singapore context. """,

#     agent=researcher,
#     tools=[tool_webscrape],
#     context = [task_search]
# )

# task_extract_findings = Task(
#     description = """Read the pdf file and return the audit findings pertaining to the {topic} and pass it to the auditor""",
#     expected_output= """A list of audit findings pertainint to the {topic}""",
#     tools=[tool_pdf],
#     agent = Audit_asssitant
# )

# task_write= Task(
#     description="""\
#     1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
#     2. Sections are properly named in an engaging manner.
#     3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

#     expected_output="""
#     A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
#     agent=auditor,
#     context=[task_websitesearch, task_extract_findings],
#     output_file="APM.md"
# )




In [7]:
auditor = Agent(
    role = "auditor",
    goal="Return the Audit Planning Memo on {topic} based on inputs from researcher and past audit findings from the audit assistant.",
    backstory = """You are an auditor. You need to consider the inputs from the audit assistant to determine what to write for the audit planning memo and risk assessment""",
    allow_delegation = True,
    max_iter = 15,
    verbose = True,
    llm=llm
)

Audit_assistant = Agent(
    role = "Audit assistant",
    goal ="From the file provided, extract the audit findings pertaining to {topic} only. Pass the audit findings to auditor",
    backstory = "You are an audit assistant. Your job is to extract the past audit findings relating to the {topic} and pass it to the auditor.",
    allow_delegation=False,
    verbose=True,
    max_iter = 15,
    llm=llm
)


task_extract_findings = Task(
    description = """Read the pdf file and return the audit findings pertaining to the {topic} and pass it to the auditor""",
    expected_output= """A list of audit findings pertainint to the {topic}""",
    tools=[tool_pdf],
    agent = Audit_assistant
)

task_write= Task(
    description="""\
    1. Use the content to craft an audit planning memo and risk assessment based on {topic} .
    2. Sections are properly named in an engaging manner.
    3. Proofread for grammatical errors and alignment with the common style used in audit planning memos.""",

    expected_output="""
    A well-written audit planning memo and risk assessment in markdown format. Each section should have minimum of 2 and maximum of 5 paragraphs.""",
    agent=auditor,
    context=[task_extract_findings],
    output_file="APM.md"
)




In [8]:
#creating the crew

crew = Crew(
    agents=[auditor,Audit_assistant],
    tasks=[task_extract_findings,task_write],
    verbose=True
)


In [ ]:
result = crew.kickoff(inputs={"topic": topic})

# Agent: Audit assistant
## Task: Read the pdf file and return the audit findings pertaining to the baby bonus and pass it to the auditor


# Agent: Audit assistant
## Thought: To find the audit findings pertaining to the baby bonus, I will search the PDF document for the term "baby bonus."
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"baby bonus\"}"
## Tool Output: 
Relevant Content:
Government CDA Withdrawals First Step Grant $231.1 million Government Co-matching Grant $294.6 millionTotal Amount $1.49 billion Total Transactions 4.42 million Number of Approved Institutions as at 11 September 2023 4,848Baby Bonus Scheme – CDA Benefits (1 April 2021 to 30 June 2023)

95Part III: Thematic Audit Schemes Eligibility Criteria25Benefits Government- Paid Paternity Leave (GPPL)/ Government- Paid Paternity Benefit (GPPB)(a) Father was lawfully married to the child’s mother at some point between conception and child’s date of birth, or within 12 months commencing on the ch

In [ ]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")